In [1]:
import pandas as pd
import xmltodict

In [2]:
with open("data/eric2021_04.xml") as fd:
    dict = xmltodict.parse(fd.read())

In [3]:
recs = [rec['metadata'] for rec in dict['records']['record']]

In [4]:
df = pd.DataFrame(recs)
df

,dcterms:accessRights,dc:subject,dc:creator,dc:type,eric:issn,dc:language,dcterms:educationLevel,dc:description,dc:identifier,dc:title,...,eric:contract_number,eric:wwcguide_link,eric:ies_publication_link,eric:ies_datasource_link,eric:note,eric:keywords_geo,eric:ies_cited,eric:wwc_reviewed,eric:keywords_law,eric:keywords_test
0,Yes,"[Cognitive Processes, Concept Formation, Conce...","[{'@scheme': 'personal author', '#text': 'Murd...",None,None,None,None,The effects and interactions of 3 variables on...,"[{'@scheme': 'eric_accno', '#text': 'ED054482'...",Concept Learning and Retention: Effects of Pre...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
1,Yes,"[Accident Prevention, Federal Legislation, Gui...","{'@scheme': 'institution', '#text': 'National ...",None,None,None,None,This pamphlet provides useful tips to parents ...,"[{'@scheme': 'eric_accno', '#text': 'ED057905'...",Safe Toys for Your Child: How to Select Them--...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
2,Yes,"[Academic Education, Auto Mechanics, Certifica...","{'@scheme': 'institution', '#text': 'National ...",Guides - Classroom - Teacher,None,English,None,This publication provides the evaluation polic...,"[{'@scheme': 'eric_accno', '#text': 'ED391005'...",ASE Program Certification Standards for Automo...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
3,Yes,"[Academic Achievement, Action Research, Algebr...","[{'@scheme': 'personal author', '#text': 'Anli...",Reports - Evaluative,None,English,None,This action research project examined the impa...,"[{'@scheme': 'eric_accno', '#text': 'ED415022'...",Improving Student Achievement through Encourag...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
4,Yes,"[Action Research, Assignments, Change Strategi...","[{'@scheme': 'personal author', '#text': 'Baco...","[Dissertations/Theses, Reports - Evaluative]",None,English,None,Problems in students completing homework may b...,"[{'@scheme': 'eric_accno', '#text': 'ED422115'...",Instructional Techniques To Improve Homework C...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Yes,"[Ethics, Counseling Services, School Counselor...","[{'@scheme': 'personal author', '#text': 'Cunn...","[Journal Articles, Reports - Descriptive]",None,English,Elementary Secondary Education,This article reviews 20 uploadable iPads appli...,"[{'@scheme': 'eric_accno', '#text': 'EJ998935'...",iPads for School Counselors: Productivity and ...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
5286,No,"[Play Therapy, Psychotherapy, School Counselor...","[{'@scheme': 'personal author', '#text': 'McBr...","[Journal Articles, Reports - Descriptive]",None,English,Middle Schools,School counselors are master jugglers and must...,"[{'@scheme': 'eric_accno', '#text': 'EJ998936'...",Integrating Sand Tray and Solution Focused Bri...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
5287,Yes,"[Best Practices, School Counselors, School Cou...","[{'@scheme': 'personal author', '#text': 'Gold...","[Journal Articles, Reports - Descriptive]",None,English,Elementary Secondary Education,Clinical supervision is a way for counselors i...,"[{'@scheme': 'eric_accno', '#text': 'EJ998939'...",Clinical Supervision Strategies for School Cou...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
5288,No,"[English Language Learners, English (Second La...","[{'@scheme': 'personal author', '#text': 'Pete...","[Journal Articles, Reports - Descriptive]",ISSN-1046-6819,English,Elementary Secondary Education,Students with learning disabilities have signi...,"[{'@scheme': 'eric_accno', '#text': 'EJ999688'...",Math and Science Strategies for English Learne...,...,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN


In [5]:
def extract_authors(dc):
    if type(dc) == list:
        return [i.get('#text') for i in dc if i.get('#text') is not None and i.get('@scheme') != "institution"]
    elif dc.get('#text') is not None and dc.get('@scheme') != "institution":
        return dc.get('#text')

In [6]:
df['authors'] = df.apply(lambda row: extract_authors(row['dc:creator']), axis=1)

In [24]:
list1 = df['authors'].tolist()
list2 = [x for x in list1 if x is not None] # remove none
flat_list = [item for sublist in list2 for item in sublist] # create a flat list just to get count

In [26]:
df2 = pd.DataFrame({'col':flat_list})
df2.to_csv("data/test.csv")